In [1]:
import numpy as np
import pandas as pd

In [25]:
class KalmanFilter:
    def __init__(self):
        self.__shape = None # [n, m], a list of state vector length n and measurment vector length m.
        self.__A = None # State transition model. Ax[t-1] = x[t]
        self.__Q = None # State transition noise covariance.
        self.__H = None # State to measurement model. Hx = z
        self.__R = None # Measurement noise covariance
    # KF shape
    @property
    def shape(self):
        return self.__shape
    @shape.setter
    def shape(self,shape_new):
        if not isinstance(shape_new,list):
            raise TypeError('New shape should be a list!')
        if not len(shape_new) == 2:
            raise ValueError('New shape should have length of 2!')
        self.__shape = shape_new    
    
    # KF Parameters
    @property
    def A(self):
        return self.__A
    @A.setter
    def A(self, A_new):
        if self.__shape is None:
            raise Exception('Shape must be set first!')
        self.__A = A_new
        return None
    @property
    def Q(self):
        return self.__Q
    @Q.setter
    def Q(self,Q_new):
        if self.__shape is None:
            raise Exception('Shape must be set first!')
        self.__Q = Q_new
        return None
    @property
    def H(self):
        return self.__H
    @H.setter
    def H(self,H_new):
        if self.__shape is None:
            raise Exception('Shape must be set first!')
        self.__H = H_new
        return None
    @property
    def R(self):
        return self.__R
    @R.setter
    def R(self,R_new):
        if self.__shape is None:
            raise Exception('Shape must be set first!')
        return None

In [28]:
foo = KalmanFilter()
foo.A = [0,1]

Exception: Shape must be set first!

In [29]:
foo = KalmanFilter()
foo.shape = [1,2]
foo.A = [0,1]